In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta

import re
import requests
import pandas as pd
import numpy as np
import time
import os

In [2]:
## 상장폐지일 기준 3개월전에는 상장폐지 확정기사가 발생
## 확정기사를 부도예측에 사용할 경우 부도예측의 오류 발생
## 따라서 (상폐일기준-3개월)을 기준으로 1년간의 데이터를 수집한다.
## 네이버 url에는 검색 시작 종료날짜가 필요한데 입력 형식이 다르기때문에 연 월 일 단위로 분리시켜놓는다.
상폐기업 = pd.read_csv('./datasets/상폐기업크롤링용리스트.csv')
상폐기업['폐지일자'] = pd.to_datetime(상폐기업['폐지일자'])

dif_3m = relativedelta(months=3)
dif_12m = relativedelta(months=12)

상폐기업['기사종료시작날짜'] = (상폐기업['폐지일자'].apply(lambda x : x - dif_3m))
종료리스트 = 상폐기업['기사종료시작날짜'].astype(str).str.split('-')
상폐기업['종료연도'] = 종료리스트.str.get(0)
상폐기업['종료월'] = 종료리스트.str.get(1)
상폐기업['종료일'] = 종료리스트.str.get(2)

상폐기업['기사시작시작날짜'] = (상폐기업['기사종료시작날짜'].apply(lambda x : x - dif_12m))
시작리스트 = 상폐기업['기사시작시작날짜'].astype(str).str.split('-')
상폐기업['시작연도'] = 시작리스트.str.get(0)
상폐기업['시작월'] = 시작리스트.str.get(1)
상폐기업['시작일'] = 시작리스트.str.get(2)

상폐기업


,회사명,폐지일자,기사종료시작날짜,종료연도,종료월,종료일,기사시작시작날짜,시작연도,시작월,시작일
0,네이쳐글로벌,2011-01-08,2010-10-08,2010,10,08,2009-10-08,2009,10,08
1,네프로아이티,2011-11-05,2011-08-05,2011,08,05,2010-08-05,2010,08,05
2,뉴젠아이씨티,2011-04-29,2011-01-29,2011,01,29,2010-01-29,2010,01,29
3,트루아워,2011-10-25,2011-07-25,2011,07,25,2010-07-25,2010,07,25
4,디패션,2011-05-07,2011-02-07,2011,02,07,2010-02-07,2010,02,07
...,...,...,...,...,...,...,...,...,...,...
130,위너지스,2018-10-11,2018-07-11,2018,07,11,2017-07-11,2017,07,11
131,위노바,2018-03-09,2017-12-09,2017,12,09,2016-12-09,2016,12,09
132,트레이스,2018-10-11,2018-07-11,2018,07,11,2017-07-11,2017,07,11
133,레이젠,2018-10-11,2018-07-11,2018,07,11,2017-07-11,2017,07,11


### 완성본

In [9]:
column_ = ['기업', '기사발행일', '기사제목', '뉴스기사본문']
뉴스_df = pd.DataFrame(columns=column_)

driver = webdriver.Chrome()

회사명_ = 상폐기업['회사명']
종료연도_ = 상폐기업['종료연도']
종료월_ = 상폐기업['종료월']
종료일_ = 상폐기업['종료일']
시작연도_ = 상폐기업['시작연도']
시작월_ = 상폐기업['시작월']
시작일_ = 상폐기업['시작일']

# start = '2010.01.01'
# end = '2010.12.31'
# start_= '20100101'
# end_ = '20101231'

for 기업, 종료연도, 종료월, 종료일, 시작연도, 시작월, 시작일 in  zip(회사명_, 종료연도_, 종료월_, 종료일_, 시작연도_, 시작월_, 시작일_):

    start = (시작연도 + '.' + 시작월 + '.' + 시작일)
    end = (종료연도 + '.' + 종료월 + '.' + 종료일)
    start_= (시작연도 + 시작월 + 시작일)
    end_ = (종료연도 + 종료월 + 종료일)

    # 나중에 뉴스_df와 concat할 임시 df 생성 및 임시 리스트들 생성
    column_ = ['기업', '기사발행일', '기사제목', '뉴스기사본문']
    임시_df = pd.DataFrame(columns=column_)    

    # 임시_df에 들어갈 리스트 생성
    본문리스트 = []
    날짜리스트 = []
    제목리스트 = []
    기업이름 = []

    # while 종료 조건으로 쓸 리스트 생성
    newslist = []
    datelist = []
    
    page = 1

    # 페이지수가 나와있지않으므로 맨끝에 page에 10씩더해서 계속 다음페이지로 이동
    # 20페이지 이후로는 광고 등 불필요한 기사가 많아 페이지를 20페이지로 한정한다.
    while page < 200:

        url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='+'"'+기업+'"'+'&sort=0&photo=0&field=0&pd=3&ds='+start+'&de='+end+'&cluster_rank=19&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from'+start_+'to'+end_+',a:all&start='+str(page)
        driver.get(url)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")

        # a태그중에서 class가 info인 것과 span태그에 class가 info 인 것 가져옴 
        news_titles = soup.select("a.info")
        dates = soup.select('span.info')

        # 네이버기사와 신문사기사 둘다 하이퍼링크가 있을경우는 기사당 news가 2개씩 네이버 기사가 없을 경우는 1개씩만 링크가 추출된다
        # 어림잡아 news_titles수가 10개 미만일 경우는 기사의 수가 5개에서 10개 미만이라는 뜻으로 해석하여 10개 이상만 링크와 기사날짜를 추출
        if len(news_titles) >= 10:

            for news in news_titles:
                title = news.attrs['href']
                newslist.append(title)

            for date in dates:
                news_date = date.text
                datelist.append(news_date)

            # 네이버 검색시 page number가 다음페이지로 갈때마다 1, 11, 21 이렇게 10씩 더해지는데 다음페이지가 없을경우 
            # 마지막 기사만을 포함한 같은 페이지를 계속 반환함. 따라서 기사와 날짜 둘 다 중복되서 저장될경우 종료함
            if (newslist[-1]==newslist[-2]) & (datelist[-1]==datelist[-2]):
                break

            page += 10

            time.sleep(1)
        
        # 기사수 5개 미만시 break, 기사없는기업 리스트에 저장함
        else:
            
            break
    
    # sid=101은 네이버 경제기사를 의미하는 듯함. 경제기사만 추출
    newslist = [news for news in newslist if 'sid=101' in news]
    
    # news_titles 뉴스기사 url 리스트가 존재시
    if news_titles:

        # 뉴스기사 url 자체에서는 text가 안가져와지는 특이사항 발생, 찾아보니 네이버기사는 인터넷에 떠야 페이지가 작동하는? 방식이라함
        # 셀레니움을 통해서 뉴스기사 url 주소로 창을 띄움
        for news in newslist:
            url = news
            driver.get(url)
            
            # 뉴스기사 url에서 본문과 제목, 기사작성날짜를 리스트에 저장함
            # 데이터프레임에 직접 행을 지정해주기는 번거로움..
            try:

                날짜 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
                날짜리스트.append(날짜)

                제목 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[2]/h2').text
                제목리스트.append(제목)

                본문 = driver.find_element('xpath', '//*[@id="dic_area"]').text
                본문리스트.append(본문)
                time.sleep(1)

                기업이름.append(기업)
            
            ## 로딩이 안되서 데이터를 못가져올 경우를 대비해 sleep 3초 주고 다시 시도
            except:

                time.sleep(3)

                try:

                    날짜 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
                    날짜리스트.append(날짜)

                    제목 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[2]/h2').text
                    제목리스트.append(제목)

                    본문 = driver.find_element('xpath', '//*[@id="dic_area"]').text
                    본문리스트.append(본문)
                    time.sleep(1)

                    기업이름.append(기업)
                
                # 그래도 데이터를 가져오지 못하는 경우는 페이지에 문제가 있다고 판단하여 PASS
                except:

                    pass

    # 혹시나 뉴스기사 url 리스트가 없을 경우는 pass                
    else:
        pass

    # 기업별로 가져온 날짜와 본문, 제목, 기업이름을 임시 데이터프레임으로 저장    
    임시_df.기사발행일 = 날짜리스트
    임시_df.뉴스기사본문 = 본문리스트
    임시_df.기사제목 = 제목리스트
    임시_df.기업 = 기업이름
    
    # 임시 데이터프레임을 뉴스 데이터프레임에 아래로 결합
    뉴스_df = pd.concat([뉴스_df, 임시_df])

# 앞에서 while 종료 조건이 같은 기사 2번저장인데 이럴 경우 중복으로 저장이 되야 종료되기때문에
# 중복기사 행 제거
뉴스_df.drop_duplicates(inplace=True)
뉴스_df.to_csv('../Step3_1_뉴스전처리/news/상폐기업뉴스.csv', index=None)


C:\Users\dbswo\AppData\Local\Temp\ipykernel_5368\1662755364.py:96: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  날짜 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
C:\Users\dbswo\AppData\Local\Temp\ipykernel_5368\1662755364.py:99: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  제목 = driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[2]/h2').text
C:\Users\dbswo\AppData\Local\Temp\ipykernel_5368\1662755364.py:102: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  본문 = driver.find_element_by_xpath('//*[@id="dic_area"]').text
C:\Users\dbswo\AppData\Local\Temp\ipykernel_5368\1662755364.py:96: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  날짜 = driver.find_element_by_xpath('//*[@id